# Code to replicate analysis
## 1. Setup
1. Mounting Google Drive & setting paths
2. Installing necessary packages
3. Downloading files for analysis


## 2. Analysis and plots
1. Diurnal gene expression (JTK_Cycle)

# 1. Setup

## 1.1 Mounting Google Drive & setting paths

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!rm -rf /content/sample_data

drive_path = '/content/drive/My Drive/' # amend this line to link to the desired path for this project

Mounted at /content/drive


## 1.2 Installing necessary packages

In [2]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
%load_ext rpy2.ipython


## 1.3 Downloading files

In [33]:
# Downloads necessary files to perform analyses (only need to be done once)
# https://github.com/emmanuel-tan/diurnal.plant.tools-2024/raw/main/NewSpeciesExpression.zip
!wget "https://github.com/emmanuel-tan/diurnal.plant.tools-2024/raw/main/NewSpeciesExpression.zip" -O diurnal_expression.zip

dir_path = drive_path + 'diurnal/'
dir_path = dir_path.replace(' ', '\ ')
!mkdir $dir_path
!unzip diurnal_expression.zip -d $dir_path
dir_path = dir_path.replace('\ ', ' ')
expmat_path = dir_path + "NewSpeciesExpression/"
!rm "diurnal_expression.zip"

--2024-04-29 23:52:29--  https://github.com/emmanuel-tan/diurnal.plant.tools-2024/raw/main/NewSpeciesExpression.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/emmanuel-tan/diurnal.plant.tools-2024/main/NewSpeciesExpression.zip [following]
--2024-04-29 23:52:29--  https://raw.githubusercontent.com/emmanuel-tan/diurnal.plant.tools-2024/main/NewSpeciesExpression.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9506216 (9.1M) [application/zip]
Saving to: ‘diurnal_expression.zip’

diurnal_expression. 100%[===================>]   9.07M  37.2MB/s    in 0.2s    

2024-04-29 23:52:30 (37.2 MB/s) 

# 2. Analysis and plots

## 2.1 Diurnal gene expression (JTK_Cycle)

In [4]:
jtk_dir = dir_path + "JTK/"
!mkdir $jtk_dir
!cd jtk_dir
!wget -P $jtk_dir "https://github.com/loubya/Mimulus_SWC_JTK_Cycle/raw/master/JTK_CYCLEv3.1.R"
%cd $jtk_dir

mkdir: cannot create directory ‘/content/drive/My’: Operation not supported
mkdir: cannot create directory ‘Drive/diurnal/JTK/’: No such file or directory
/bin/bash: line 1: cd: jtk_dir: No such file or directory
--2024-04-29 23:39:44--  http://drive/diurnal/JTK/
Resolving drive (drive)... failed: No address associated with hostname.
wget: unable to resolve host address ‘drive’
--2024-04-29 23:39:44--  https://github.com/loubya/Mimulus_SWC_JTK_Cycle/raw/master/JTK_CYCLEv3.1.R
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/loubya/Mimulus_SWC_JTK_Cycle/master/JTK_CYCLEv3.1.R [following]
--2024-04-29 23:39:44--  https://raw.githubusercontent.com/loubya/Mimulus_SWC_JTK_Cycle/master/JTK_CYCLEv3.1.R
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecti

In [5]:
# import paths for R to use as variables (remove backslashes)
jtk_dir_R = jtk_dir.replace("\ ", " ")
dir_path_R = dir_path.replace("\ ", " ")
expmat_path_R = expmat_path.replace("\ ", " ")

In [6]:
%%R -i jtk_dir_R -i dir_path_R -i expmat_path_R

NULL


### 2.1.1 *Brassica rapa*

In [7]:
%%R
source("JTK_CYCLEv3.1.R")

project <- "BRA_JTK"

options(stringsAsFactors=FALSE)
annot <- read.delim(paste(expmat_path_R, "BRA_annot.txt", sep=""))
data <- read.delim(paste(expmat_path_R, "BRA_expmat.txt", sep=""))

rownames(data) <- data[,1]
data <- data[,-1]
jtkdist(4, 2)

periods <- 6:6
jtk.init(periods,8)

cat("JTK analysis started on",date(),"\n")
flush.console()

st <- system.time({
  res <- apply(data,1,function(z) {
    jtkx(z)
    c(JTK.ADJP,JTK.PERIOD,JTK.LAG,JTK.AMP)
  })
  res <- as.data.frame(t(res))
  bhq <- p.adjust(unlist(res[,1]),"BH")
  res <- cbind(bhq,res)
  colnames(res) <- c("BH.Q","ADJ.P","PER","LAG","AMP")
  results <- cbind(annot,res,data)
  results <- results[order(res$ADJ.P,-res$AMP),]
})
print(st)

save(results,file=paste(project,"rda",sep="."))
write.table(results,file=paste(project,"txt",sep="."),row.names=F,col.names=T,quote=F,sep="\t")

JTK analysis started on Mon Apr 29 23:39:46 2024 
   user  system elapsed 
 27.454   0.025  27.861 


In [8]:
# clean JTK output
jtk_output_path = jtk_dir + "BRA_JTK.txt"
jtk_output = pd.read_csv(jtk_output_path, sep='\t')
jtk_output = jtk_output.set_index('Probe')

jtk_clean = jtk_output

# get just expression data portion
expmat = jtk_output.iloc[:,7:]

# get NE genes
NE_genes = []
for gene, row, in expmat.iterrows():
    if row.max() < 1: NE_genes.append(gene)

# assign "NE" to ADJ.P and LAG column
for gene, row in jtk_clean.iterrows():
    if gene in NE_genes:
        jtk_clean.at[gene, 'ADJ.P'] = "NE"
        jtk_clean.at[gene, 'LAG'] = "NE"

# remove unwanted columns
unwanted_cols = ['BH.Q', 'PER', 'AMP']
original_cols = jtk_clean.columns
new_columns = [col for col in original_cols if col not in unwanted_cols]
jtk_clean = jtk_clean[new_columns]

# assign "NR" to ADJ.P and LAG column
for gene, row in jtk_clean.iterrows():
    if type(row['ADJ.P']) is float:
        if row['ADJ.P'] > 0.05:
            jtk_clean.at[gene, 'ADJ.P'] = "NR"
            jtk_clean.at[gene, 'LAG'] = "NR"

jtk_clean = jtk_clean.sort_index()

# save file
jtk_clean.to_csv(jtk_dir + "BRA_JTK_clean.txt", sep='\t')

### 2.1.2 *Hordeum vulagre*

In [9]:
%%R
source("JTK_CYCLEv3.1.R")

project <- "HVU_JTK"

options(stringsAsFactors=FALSE)
annot <- read.delim(paste(expmat_path_R, "HVU_annot.txt", sep=""))
data <- read.delim(paste(expmat_path_R, "HVU_expmat.txt", sep=""))

rownames(data) <- data[,1]
data <- data[,-1]
jtkdist(7, 2)

periods <- 6:6
jtk.init(periods,4)

cat("JTK analysis started on",date(),"\n")
flush.console()

st <- system.time({
  res <- apply(data,1,function(z) {
    jtkx(z)
    c(JTK.ADJP,JTK.PERIOD,JTK.LAG,JTK.AMP)
  })
  res <- as.data.frame(t(res))
  bhq <- p.adjust(unlist(res[,1]),"BH")
  res <- cbind(bhq,res)
  colnames(res) <- c("BH.Q","ADJ.P","PER","LAG","AMP")
  results <- cbind(annot,res,data)
  results <- results[order(res$ADJ.P,-res$AMP),]
})
print(st)

save(results,file=paste(project,"rda",sep="."))
write.table(results,file=paste(project,"txt",sep="."),row.names=F,col.names=T,quote=F,sep="\t")



JTK analysis started on Mon Apr 29 23:40:44 2024 
   user  system elapsed 
 32.891   0.061  33.599 


In [10]:
# clean JTK output
jtk_output_path = jtk_dir + "HVU_JTK.txt"
jtk_output = pd.read_csv(jtk_output_path, sep='\t')
jtk_output = jtk_output.set_index('Probe')

jtk_clean = jtk_output

# get just expression data portion
expmat = jtk_output.iloc[:,7:]

# get NE genes
NE_genes = []
for gene, row, in expmat.iterrows():
    if row.max() < 1: NE_genes.append(gene)

# assign "NE" to ADJ.P and LAG column
for gene, row in jtk_clean.iterrows():
    if gene in NE_genes:
        jtk_clean.at[gene, 'ADJ.P'] = "NE"
        jtk_clean.at[gene, 'LAG'] = "NE"

# remove unwanted columns
unwanted_cols = ['BH.Q', 'PER', 'AMP']
original_cols = jtk_clean.columns
new_columns = [col for col in original_cols if col not in unwanted_cols]
jtk_clean = jtk_clean[new_columns]

# assign "NR" to ADJ.P and LAG column
for gene, row in jtk_clean.iterrows():
    if type(row['ADJ.P']) is float:
        if row['ADJ.P'] > 0.05:
            jtk_clean.at[gene, 'ADJ.P'] = "NR"
            jtk_clean.at[gene, 'LAG'] = "NR"

jtk_clean = jtk_clean.sort_index()
jtk_clean.to_csv(jtk_dir + "HVU_JTK_clean.txt", sep='\t')

### 2.1.3 *Marchantia polymorpha*

In [11]:
%%R
source("JTK_CYCLEv3.1.R")

project <- "MPO_JTK"

options(stringsAsFactors=FALSE)
annot <- read.delim(paste(expmat_path_R, "MPO_annot.txt", sep=""))
data <- read.delim(paste(expmat_path_R, "MPO_expmat.txt", sep=""))

rownames(data) <- data[,1]
data <- data[,-1]
jtkdist(6, 3)

periods <- 6:6
jtk.init(periods,4)

cat("JTK analysis started on",date(),"\n")
flush.console()

st <- system.time({
  res <- apply(data,1,function(z) {
    jtkx(z)
    c(JTK.ADJP,JTK.PERIOD,JTK.LAG,JTK.AMP)
  })
  res <- as.data.frame(t(res))
  bhq <- p.adjust(unlist(res[,1]),"BH")
  res <- cbind(bhq,res)
  colnames(res) <- c("BH.Q","ADJ.P","PER","LAG","AMP")
  results <- cbind(annot,res,data)
  results <- results[order(res$ADJ.P,-res$AMP),]
})
print(st)

save(results,file=paste(project,"rda",sep="."))
write.table(results,file=paste(project,"txt",sep="."),row.names=F,col.names=T,quote=F,sep="\t")

JTK analysis started on Mon Apr 29 23:41:54 2024 
   user  system elapsed 
  9.355   0.034   9.812 


In [12]:
# clean JTK output
jtk_output_path = jtk_dir + "MPO_JTK.txt"
jtk_output = pd.read_csv(jtk_output_path, sep='\t')
jtk_output = jtk_output.set_index('Probe')

jtk_clean = jtk_output

# get just expression data portion
expmat = jtk_output.iloc[:,7:]

# get NE genes
NE_genes = []
for gene, row, in expmat.iterrows():
    if row.max() < 1: NE_genes.append(gene)

# assign "NE" to ADJ.P and LAG column
for gene, row in jtk_clean.iterrows():
    if gene in NE_genes:
        jtk_clean.at[gene, 'ADJ.P'] = "NE"
        jtk_clean.at[gene, 'LAG'] = "NE"

# remove unwanted columns
unwanted_cols = ['BH.Q', 'PER', 'AMP']
original_cols = jtk_clean.columns
new_columns = [col for col in original_cols if col not in unwanted_cols]
jtk_clean = jtk_clean[new_columns]

# assign "NR" to ADJ.P and LAG column
for gene, row in jtk_clean.iterrows():
    if type(row['ADJ.P']) is float:
        if row['ADJ.P'] > 0.05:
            jtk_clean.at[gene, 'ADJ.P'] = "NR"
            jtk_clean.at[gene, 'LAG'] = "NR"

jtk_clean = jtk_clean.sort_index()

# adjust lag by 2.0
def adjustLag(lag):
    if str(lag) == "NE":
        new_val = "NE"
    elif str(lag) == "NR":
        new_val = "NR"
    else:
        new_val = str(float(lag) + 2.0)
        if float(new_val) >= 24:
            new_val = str(float(new_val)-24)
    return new_val

jtk_clean['LAG'] = jtk_clean['LAG'].apply(adjustLag).to_list()

# save as supplementary file
jtk_clean.to_csv(jtk_dir + "MPO_JTK_clean.txt", sep='\t')

### 2.1.4 *Sorghum bicolor*

In [13]:
%%R
source("JTK_CYCLEv3.1.R")

project <- "SBI_JTK"

options(stringsAsFactors=FALSE)
annot <- read.delim(paste(expmat_path_R, "SBI_annot.txt", sep=""))
data <- read.delim(paste(expmat_path_R, "SBI_expmat.txt", sep=""))

rownames(data) <- data[,1]
data <- data[,-1]
jtkdist(8, 3)

periods <- 8:8
jtk.init(periods,3)

cat("JTK analysis started on",date(),"\n")
flush.console()

st <- system.time({
  res <- apply(data,1,function(z) {
    jtkx(z)
    c(JTK.ADJP,JTK.PERIOD,JTK.LAG,JTK.AMP)
  })
  res <- as.data.frame(t(res))
  bhq <- p.adjust(unlist(res[,1]),"BH")
  res <- cbind(bhq,res)
  colnames(res) <- c("BH.Q","ADJ.P","PER","LAG","AMP")
  results <- cbind(annot,res,data)
  results <- results[order(res$ADJ.P,-res$AMP),]
})
print(st)

save(results,file=paste(project,"rda",sep="."))
write.table(results,file=paste(project,"txt",sep="."),row.names=F,col.names=T,quote=F,sep="\t")

JTK analysis started on Mon Apr 29 23:42:13 2024 
   user  system elapsed 
 35.715   0.079  38.150 


In [14]:
# clean JTK output
jtk_output_path = jtk_dir + "SBI_JTK.txt"
jtk_output = pd.read_csv(jtk_output_path, sep='\t')
jtk_output = jtk_output.set_index('Probe')

jtk_clean = jtk_output

# get just expression data portion
expmat = jtk_output.iloc[:,7:]

# get NE genes
NE_genes = []
for gene, row, in expmat.iterrows():
    if row.max() < 1: NE_genes.append(gene)

# assign "NE" to ADJ.P and LAG column
for gene, row in jtk_clean.iterrows():
    if gene in NE_genes:
        jtk_clean.at[gene, 'ADJ.P'] = "NE"
        jtk_clean.at[gene, 'LAG'] = "NE"

# remove unwanted columns
unwanted_cols = ['BH.Q', 'PER', 'AMP']
original_cols = jtk_clean.columns
new_columns = [col for col in original_cols if col not in unwanted_cols]
jtk_clean = jtk_clean[new_columns]

# assign "NR" to ADJ.P and LAG column
for gene, row in jtk_clean.iterrows():
    if type(row['ADJ.P']) is float:
        if row['ADJ.P'] > 0.05:
            jtk_clean.at[gene, 'ADJ.P'] = "NR"
            jtk_clean.at[gene, 'LAG'] = "NR"

jtk_clean = jtk_clean.sort_index()

# adjust lag by 3.0
def adjustLag(lag):
    if str(lag) == "NE":
        new_val = "NE"
    elif str(lag) == "NR":
        new_val = "NR"
    else:
        new_val = str(float(lag) + 3.0)
        if float(new_val) >= 24:
            new_val = str(float(new_val)-24)
    return new_val

jtk_clean['LAG'] = jtk_clean['LAG'].apply(adjustLag).to_list()

# save as supplementary file
jtk_clean.to_csv(jtk_dir + "SBI_JTK_clean.txt", sep='\t')

### 2.1.5 *Setaria italica*

In [15]:
%%R
source("JTK_CYCLEv3.1.R")

project <- "SIT_JTK"

options(stringsAsFactors=FALSE)
annot <- read.delim(paste(expmat_path_R, "SIT_annot.txt", sep=""))
data <- read.delim(paste(expmat_path_R, "SIT_expmat.txt", sep=""))

rownames(data) <- data[,1]
data <- data[,-1]
jtkdist(8, 3)

periods <- 8:8
jtk.init(periods,3)

cat("JTK analysis started on",date(),"\n")
flush.console()

st <- system.time({
  res <- apply(data,1,function(z) {
    jtkx(z)
    c(JTK.ADJP,JTK.PERIOD,JTK.LAG,JTK.AMP)
  })
  res <- as.data.frame(t(res))
  bhq <- p.adjust(unlist(res[,1]),"BH")
  res <- cbind(bhq,res)
  colnames(res) <- c("BH.Q","ADJ.P","PER","LAG","AMP")
  results <- cbind(annot,res,data)
  results <- results[order(res$ADJ.P,-res$AMP),]
})
print(st)

save(results,file=paste(project,"rda",sep="."))
write.table(results,file=paste(project,"txt",sep="."),row.names=F,col.names=T,quote=F,sep="\t")

JTK analysis started on Mon Apr 29 23:43:16 2024 
   user  system elapsed 
 38.496   0.052  39.716 


In [16]:
# clean JTK output
jtk_output_path = jtk_dir + "SIT_JTK.txt"
jtk_output = pd.read_csv(jtk_output_path, sep='\t')
jtk_output = jtk_output.set_index('Probe')

jtk_clean = jtk_output

# get just expression data portion
expmat = jtk_output.iloc[:,7:]

# get NE genes
NE_genes = []
for gene, row, in expmat.iterrows():
    if row.max() < 1: NE_genes.append(gene)

# assign "NE" to ADJ.P and LAG column
for gene, row in jtk_clean.iterrows():
    if gene in NE_genes:
        jtk_clean.at[gene, 'ADJ.P'] = "NE"
        jtk_clean.at[gene, 'LAG'] = "NE"

# remove unwanted columns
unwanted_cols = ['BH.Q', 'PER', 'AMP']
original_cols = jtk_clean.columns
new_columns = [col for col in original_cols if col not in unwanted_cols]
jtk_clean = jtk_clean[new_columns]

# assign "NR" to ADJ.P and LAG column
for gene, row in jtk_clean.iterrows():
    if type(row['ADJ.P']) is float:
        if row['ADJ.P'] > 0.05:
            jtk_clean.at[gene, 'ADJ.P'] = "NR"
            jtk_clean.at[gene, 'LAG'] = "NR"

jtk_clean = jtk_clean.sort_index()

# adjust lag by 3.0
def adjustLag(lag):
    if str(lag) == "NE":
        new_val = "NE"
    elif str(lag) == "NR":
        new_val = "NR"
    else:
        new_val = str(float(lag) + 3.0)
        if float(new_val) >= 24:
            new_val = str(float(new_val)-24)
    return new_val

jtk_clean['LAG'] = jtk_clean['LAG'].apply(adjustLag).to_list()

# save as supplementary file
jtk_clean.to_csv(jtk_dir + "SIT_JTK_clean.txt", sep='\t')